<a href="https://colab.research.google.com/github/Devabubakar/Deep-Learning-with-Keras/blob/main/Keras.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Assignment 1 - Deep Learning with Keras

This assignment will test you on the following skills: implementing a convolutional neural network in Keras, utilising the Functional Model API, building custom layers and loss functions, and analysing a trained model.

The model you will be building is called a Variational Autoencoder (VAE), a special type of neural network that uses Bayesian Inference to generate synthetic data. As described in the lectures, a VAE compresses a sample input into a low dimensional space (a latent vector). A constraint is applied in the form of a modification to the loss function, which has the effect of forcing the latent vector to look like a standard normal distribution.

This notebook is divided into sections, which you should use to complete the following tasks:

1. Implement a custom layer that performs the "reparameterization trick" described in the lectures.
2. Use the functional model API in keras to create an encoder model and a decoder model, using the specifications provided. Combine these models into the full VAE architecture.
3. Train the model using the celeb_a dataset of celebrity faces.
4. Create plots that demonstrate the ability of the network to reconstruct images, and generate new images.

In addition to submitting your completed notebook, you should write a 1-page report that discusses the results of the model.

# Module imports




In [24]:
import tensorflow as tf
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [25]:
!mkdir /content/celeba

mkdir: cannot create directory ‘/content/celeba’: File exists


In [28]:
!unzip /content/drive/MyDrive/img_align_celeba.zip -d /content/celeba

Streaming output truncated to the last 5000 lines.
  inflating: /content/celeba/img_align_celeba/img_align_celeba/003938.jpg  
  inflating: /content/celeba/__MACOSX/img_align_celeba/img_align_celeba/._003938.jpg  
  inflating: /content/celeba/img_align_celeba/img_align_celeba/025041.jpg  
  inflating: /content/celeba/__MACOSX/img_align_celeba/img_align_celeba/._025041.jpg  
  inflating: /content/celeba/img_align_celeba/img_align_celeba/005391.jpg  
  inflating: /content/celeba/__MACOSX/img_align_celeba/img_align_celeba/._005391.jpg  
  inflating: /content/celeba/img_align_celeba/img_align_celeba/012835.jpg  
  inflating: /content/celeba/__MACOSX/img_align_celeba/img_align_celeba/._012835.jpg  
  inflating: /content/celeba/img_align_celeba/img_align_celeba/023430.jpg  
  inflating: /content/celeba/__MACOSX/img_align_celeba/img_align_celeba/._023430.jpg  
  inflating: /content/celeba/img_align_celeba/img_align_celeba/020139.jpg  
  inflating: /content/celeba/__MACOSX/img_align_celeba/img

# Task 1 - Reparameterisation layer

### Create a class called latent_sampling, which subclasses layers.Layer.
### The class should perform the reparameterisation trick in its .call()
### method.
### Reparameterization Trick: z = mean + epsilon * exp(ln(variance) * 0.5)
### epsilon = N(0,1), a unit normal with same dims as mean and variance

# Include the follow two lines in your .call method:
 

```
self.add_loss(-0.5 * tf.reduce_sum(1 + logvar - tf.square(mean) - tf.exp(logvar)))
# self.add_metric(-0.5 * tf.reduce_sum(1 + logvar - tf.square(mean) - tf.exp(logvar)), name='kl_loss')
```




In [1]:
import tensorflow as tf
from tensorflow.keras import layers

class LatentSampling(layers.Layer):
  def call(self, inputs, **kwargs):
    mean, logvar = inputs
    epsilon = tf.random.normal(shape=tf.shape(mean))
    z = mean + epsilon * tf.exp(logvar * 0.5)
    self.add_loss(-0.5 * tf.reduce_sum(1 + logvar - tf.square(mean) - tf.exp(logvar)))
    self.add_metric(-0.5 * tf.reduce_sum(1 + logvar - tf.square(mean) - tf.exp(logvar)), name='kl_loss')
    return z


# Task 2 - Model Definitions

### Create the encoder model, using the functional API and the architecture
### detailed below. Use tf.keras.models.Model to initialise the model.

Model: "encoder"
# ____________________________________________________________________________________________________
Layer (type)            Output Shape           Activation  kernel_size  padding  Input
# ====================================================================================================


```
enc_input (InputLayer)  [(None, 128, 128, 3)]  None
enc_conv_1 (Conv2D)     (None, 64, 64, 32)     ReLU        (3,3)        'same'   enc_input
enc_conv_2 (Conv2D)     (None, 32, 32, 64)     ReLU        (3,3)        'same'   enc_conv_1
enc_conv_3 (Conv2D)     (None, 16, 16, 64)     ReLU        (3,3)        'same'   enc_conv_2    
enc_conv_4 (Conv2D)     (None, 8, 8, 64)       ReLU        (3,3)        'same'   enc_conv_3    
enc_flat (Flatten)      (None, 4096)           None        None         None     enc_conv_4
z_mean (Dense)          (None, 200)            None        None         None     enc_flat                        
z_log_var (Dense)       (None, 200)            None        None         None     enc_flat
z (latent_sampling)     (None, 200)            None        None         None     (z_mean, z_log_var)

```




In [33]:
import tensorflow as tf
from tensorflow.keras import layers

# Define the input layer
encoder_input = tf.keras.Input(shape=(128, 128, 3))

# Define the encoder layers
x = layers.Conv2D(32, kernel_size=3, activation='relu', padding='same')(encoder_input)
x = layers.Conv2D(64, kernel_size=3, activation='relu', padding='same')(x)
x = layers.Conv2D(64, kernel_size=3, activation='relu', padding='same')(x)
x = layers.Conv2D(64, kernel_size=3, activation='relu', padding='same')(x)
x = layers.Flatten()(x)
z_mean = layers.Dense(200)(x)
z_log_var = layers.Dense(200)(x)

# Use the latent_sampling layer to perform the reparameterization trick
z = LatentSampling()([z_mean, z_log_var])

# Initialize the encoder model
encoder = tf.keras.Model(encoder_input, [z_mean, z_log_var, z], name='encoder')


### Create the decoder model, using the functional API and the architecture
### detailed below. Use tf.keras.models.Model to initialise the model.

Model: "decoder"
# ____________________________________________________________________________________________________
 Layer (type)                   Output Shape           Activation  kernel_size  padding  Input
# ====================================================================================================




```
#  dec_input (InputLayer)         [(None, 200)]          None
#  dec_dense (Dense)              (None, 4096)           ReLU        None         None     dec_input
#  dec_reshape (Reshape)          (None, 8, 8, 64)       None        None         None     dec_dense
#  dec_conv_1 (Conv2DTranspose)   (None, 8, 8, 64)       ReLU        (3,3)        'same'   dec_reshape
#  dec_conv_2 (Conv2DTranspose)   (None, 16, 16, 64)     ReLU        (3,3)        'same'   dec_conv_1
#  dec_conv_3 (Conv2DTranspose)   (None, 32, 32, 64)     ReLU        (3,3)        'same'   dec_conv_2    
#  dec_conv_4 (Conv2DTranspose)   (None, 64, 64, 32)     ReLU        (3,3)        'same'   dec_conv_3    
#  dec_output (Conv2DTranspose)   (None, 128, 128, 3)    ReLU        (3,3)        'same'   dec_conv_4
```




In [4]:
import tensorflow as tf
from tensorflow.keras import layers

# Define the input layer
decoder_input = tf.keras.Input(shape=(200,))

# Define the decoder layers
x = layers.Dense(4096, activation='relu')(decoder_input)
x = layers.Reshape((8, 8, 64))(x)
x = layers.Conv2DTranspose(64, kernel_size=3, activation='relu', padding='same')(x)
x = layers.Conv2DTranspose(64, kernel_size=3, activation='relu', padding='same')(x)
x = layers.Conv2DTranspose(64, kernel_size=3, activation='relu', padding='same')(x)
x = layers.Conv2DTranspose(32, kernel_size=3, activation='relu', padding='same')(x)
decoder_output = layers.Conv2DTranspose(3, kernel_size=3, activation='relu', padding='same')(x)

# Initialize the decoder model
decoder = tf.keras.Model(decoder_input, decoder_output, name='decoder')


### Create the VAE model, again using tf.keras.models.Model, with the function
### API to combine the feed the outputs of the encoder into the inputs of the
### decoder.

In [5]:
import tensorflow as tf
from tensorflow.keras import layers

# Define the input layer for the VAE model
vae_input = tf.keras.Input(shape=(128, 128, 3))

# Pass the input through the encoder model to get the latent variables
z_mean, z_log_var, z = encoder(vae_input)

# Pass the latent variables through the decoder model to get the output image
output = decoder(z)

# Initialize the VAE model
vae = tf.keras.Model(vae_input, output, name='vae')


# Task 3 - Train the model

In [6]:
# Provided here are the loss functions for the VAE model.

def recon_loss(y_true, y_pred):
    recon = tf.reduce_sum(tf.square(y_true-y_pred), axis=(1,2,3))
    return tf.reduce_mean(recon)

### Compile the VAE model, choosing an appropriate optimizer and learning rate,
### the total_loss function as the model loss, and any appropriate metrics.

### Train the model using the train dataset for an appropriate number of epochs.
### Store the losses and metrics in the history dictionary.

# This code is provided to load a subsample of the celeb_a dataset, and process
# the images into the correct format for the model.

In [100]:
import tensorflow_datasets as tfds

def img_process(features):
    """
    A preprocessing fuction for the test and validation datasets. This function
    accepts the oxford_iiit_pet dataset, extracts the images and species label,
    and resizes and rescales the images.
    """
    image = tf.image.resize(features['image'], (128,128))
    image = tf.cast(image, 'float32')/255.
    return image, image

gcs_base_dir = "gs://celeb_a_dataset/"
celeb_a_builder = tfds.builder("celeb_a", data_dir=gcs_base_dir, version='2.0.0')

celeb_a_builder.download_and_prepare()

version = str(celeb_a_builder.info.version)
print('Celeb_A dataset version: %s' % version)
train_ds, test_ds = celeb_a_builder.as_dataset(split=tfds.Split.TRAIN, shuffle_files=True), celeb_a_builder.as_dataset(split=tfds.Split.TEST, shuffle_files=True)

train_ds = train_ds.map(img_process).cache().batch(64)
test_ds = test_ds.map(img_process).cache().batch(64)


Celeb_A dataset version: 2.0.0


In [101]:
import tensorflow as tf

# Train the VAE model on the celeb_a dataset
history = vae.fit(train_ds, epochs=10, validation_data=test_ds)


Epoch 1/10


ValueError: ignored